In [ ]:
import umap
import cupy as cp
from google.colab import drive
from cuml import UMAP, KMeans, DBSCAN
import os, re, json, glob, warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import groupby
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import joblib

In [ ]:
warnings.filterwarnings("ignore", category=FutureWarning)
# Path
DATA_DIR = "/content/drive/MyDrive/MSE 641 Project Data"
ONET_CSV = os.path.join(DATA_DIR, "Task_Statements_standardized.csv")
JSONL_CHAT = os.path.join(DATA_DIR, "wildchat_en_cleaned.jsonl")
METRICS_CSV = os.path.join(DATA_DIR, "bert_kmeans_dbscan_metrics_best_per_k.csv")
LABEL_GLOB = os.path.join(DATA_DIR, "bert_kmeans_labels_k*_best.pkl")
OUT_CSV = os.path.join(DATA_DIR, "onet_mapping_results.csv")
OUT_JSONL = os.path.join(DATA_DIR, "onet_mapping_results.jsonl")

TOP_K_CANDIDATES = 5  # how many O*NET candidates to show to LLM
REP_TOP_N = 5  # how many representative samples per cluster (by center similarity)

# OpenAI client or fallback
openai_api_key = os.environ.get("OPENAI_API_KEY", "")
USE_LLM = False
try:
  if not openai_api_key:
    # Try Colab Secrets if available
    from google.colab import userdata  # type: ignore
    openai_api_key = userdata.get("OPENAI_API_KEY") or ""
    if openai_api_key:
      os.environ["OPENAI_API_KEY"] = openai_api_key
except Exception:
  pass

if openai_api_key:
  try:
    from openai import OpenAI
    client = OpenAI(api_key=openai_api_key)
    LLM_MODEL = "gpt-4o-mini-2024-07-18"
    USE_LLM = True
    print("[INFO] OPENAI_API_KEY detected. LLM selection enabled.")
  except Exception as e:
    print(f"[WARN] Failed to init OpenAI client: {e}")
    print("[WARN] Proceeding WITHOUT LLM (top-1 cosine fallback).")
else:
  print("[WARN] OPENAI_API_KEY not set. Proceeding WITHOUT LLM (top-1 cosine fallback).")

# Load O*NET and embed
if not os.path.exists(ONET_CSV):
    raise FileNotFoundError(f"O*NET CSV not found: {ONET_CSV}")

onet_df = pd.read_csv(ONET_CSV)
onet_codes = onet_df["O*NET-SOC Code"].astype(str).tolist()
onet_titles = onet_df["Title"].astype(str).tolist()
onet_descs = onet_df["Task"].astype(str).tolist()
onet_texts = onet_df["task_standardized"].astype(str).tolist()

print(f"[INFO] Loaded O*NET rows: {len(onet_texts)}")

embed_model = SentenceTransformer("all-MiniLM-L6-v2")
onet_embs = embed_model.encode(onet_texts, batch_size=64, show_progress_bar=True)

# Load clustered conversations (prompt+response)
texts = []
with open(JSONL_CHAT, "r", encoding="utf-8") as f:
  for line in f:
    if not line.strip():
      continue
    obj = json.loads(line)
    if "prompt" in obj and "response" in obj:
      p = str(obj["prompt"]).strip()
      r = str(obj["response"]).strip()
      texts.append((p + " " + r).strip())
print(f"[INFO] Loaded conversations: {len(texts)}")

# Determine k values to process
if os.path.exists(METRICS_CSV):
  summary_df = pd.read_csv(METRICS_CSV)
  best_ks = sorted(pd.unique(summary_df["k"]))
else:
  label_files = glob.glob(LABEL_GLOB)
  best_ks = sorted({int(re.search(r"k(\d+)", os.path.basename(p)).group(1)) for p in label_files})
print("[INFO] k values found:", best_ks)

# Mapping
results = []

for k in tqdm(best_ks, desc="Mapping per k"):
  labels_path = os.path.join(DATA_DIR, f"bert_kmeans_labels_k{k}_best.pkl")
  if not os.path.exists(labels_path):
    print(f"[skip] labels not found for k={k}: {labels_path}")
    continue

  labels = joblib.load(labels_path)
  if len(labels) != len(texts):
    print(f"[skip] len(labels) != len(texts) for k={k}: {len(labels)} vs {len(texts)}")
    continue

  df = pd.DataFrame({"text": [t for t in texts], "label": labels})
  # deterministic order
  for cid, group in df.groupby("label", sort=True):
    all_texts = group["text"].tolist()

    # Representative samples: closest to cluster center in embedding space
    if not all_texts:
      samples = []
    else:
      text_embs = embed_model.encode(all_texts, batch_size=32, show_progress_bar=False)
      center = text_embs.mean(axis=0).reshape(1, -1)
      sims = cosine_similarity(center, text_embs)[0]
      n_pick = min(REP_TOP_N, len(all_texts))
      top_idxs = np.argsort(-sims)[:n_pick]
      samples = [all_texts[i] for i in top_idxs]
      # Deduplicate samples but keep order
      samples = [*dict.fromkeys(samples)]

    if not samples:
      results.append({
        "k": k, "Cluster": int(cid),
        "O*NET Code": None, "Title": None, "Description": None,
        "Similarity": None, "GPT Choice": "None", "Samples": samples
      })
      continue

    # Top-K O*NET by cosine similarity
    sample_vec = embed_model.encode(" ".join(samples), convert_to_numpy=True)
    scores = cosine_similarity(sample_vec.reshape(1, -1), onet_embs)[0]
    cand_idxs = np.argsort(-scores)[:TOP_K_CANDIDATES]

    if USE_LLM:
      # Build prompt for LLM disambiguation
      prompt = "Cluster samples:\n"
      for i, s in enumerate(samples, 1):
        snippet = s.replace("\n", " ")[:200]
        prompt += f"{i}. {snippet}…\n"
      prompt += f"\nTop {TOP_K_CANDIDATES} O*NET candidates:\n"
      for i, idx in enumerate(cand_idxs, 1):
        prompt += (f"{i}. Code: {onet_codes[idx]}, Title: {onet_titles[idx]}\n"
              f"Desc: {onet_descs[idx][:200]}…\n")
      prompt += (f"\nWhich single candidate (1–{TOP_K_CANDIDATES}) best matches these samples? "
                  f"If none, reply 'None'. Just answer the number or None.")

      try:
        resp = client.chat.completions.create(
          model=LLM_MODEL,
          messages=[{"role": "user", "content": prompt}],
          temperature=0.0,
          max_tokens=10,
        )
        choice = (resp.choices[0].message.content or "").strip()
      except Exception as e:
        print(f"[warn] LLM error on k={k}, cluster={cid}: {e}")
        choice = "None"

      if choice.isdigit() and 1 <= int(choice) <= TOP_K_CANDIDATES:
        sel  = cand_idxs[int(choice) - 1]
        code, title, desc, sim = onet_codes[sel], onet_titles[sel], onet_descs[sel], float(scores[sel])
      else:
        code, title, desc, sim = None, None, None, None
  else:
    # in case LLM does not, pick top-1 cosine candidate
    sel  = cand_idxs[0]
    code, title, desc, sim = onet_codes[sel], onet_titles[sel], onet_descs[sel], float(scores[sel])
    choice = "1* (no-LLM fallback)"

  results.append({
      "k": k,
      "Cluster": int(cid),
      "O*NET Code": code,
      "Title": title,
      "Description": desc,
      "Similarity": sim,
      "GPT Choice": choice,
      "Samples": samples
    })

# print per-k details
results_sorted = sorted(results, key=lambda x: (x["k"], x["Cluster"]))
for k_val, group in groupby(results_sorted, key=lambda x: x["k"]):
  print("\n" + "=" * 40)
  print(f"Results for k = {k_val}")
  print("=" * 40)
  for r in group:
    print(f"\nCluster {r['Cluster']} → {r['O*NET Code'] or 'None'}: {r.get('Title','') or 'No good match'}")
    print(f"Desc: {r.get('Description','') or ''}")
    print(f"Similarity: {r.get('Similarity','')}")
    print(f"GPT Picked: {r.get('GPT Choice','')}")
    print("Samples:")
    for i, s in enumerate(r.get("Samples", []), 1):
        snippet = s.replace("\n", " ")[:300]
        print(f"{i}. {snippet}…")

# saving (CSV + JSONL)
def _to_py_scalar(x):
  if isinstance(x, np.generic):
    return x.item()
  return x

def _sanitize_val(v):
  if isinstance(v, float) and (np.isnan(v) or np.isinf(v)):
    return None
  if isinstance(v, (np.floating, np.integer, np.bool_)):
    return v.item()
  if isinstance(v, list):
    return [_sanitize_val(x) for x in v]
  if isinstance(v, dict):
    return {k: _sanitize_val(x) for k, x in v.items()}
  return v

def _json_sanitized_records(df: pd.DataFrame):
  records = df.to_dict(orient="records")
  return [{k: _sanitize_val(v) for k, v in rec.items()} for rec in records]

# Normalize records & deduplicate by (k, Cluster)
rows  = [{k: _to_py_scalar(v) for k, v in r.items()} for r in results]
df_all = pd.DataFrame(rows)
dedup_keys = [c for c in ["k", "Cluster"] if c in df_all.columns]
if dedup_keys:
  df_all = df_all.drop_duplicates(subset=dedup_keys, keep="last").reset_index(drop=True)

# Clean records for JSONL/CSV
clean_records = _json_sanitized_records(df_all)

# Write JSONL
with open(OUT_JSONL, "w", encoding="utf-8") as f:
  for rec in clean_records:
    json.dump(rec, f, ensure_ascii=False, allow_nan=False)
    f.write("\n")

# Write CSV (stringify lists/dicts)
df_csv = pd.DataFrame(clean_records).copy()
for col in df_csv.columns:
  if df_csv[col].apply(lambda v: isinstance(v, (list, dict))).any():
    df_csv[col] = df_csv[col].apply(lambda v: json.dumps(v, ensure_ascii=False))
df_csv.to_csv(OUT_CSV, index=False)

print("\n[INFO] Saved:", OUT_CSV)
print("[INFO] Saved:", OUT_JSONL)

# Statistics
def _norm_code_series(s: pd.Series) -> pd.Series:
  s = s.astype(str).str.strip()
  return s.replace({"": np.nan, "None": np.nan, "none": np.nan, "nan": np.nan})

print("\n===== STATISTICS =====")
df_stats = pd.DataFrame(clean_records).copy()

# figure out code/title columns
code_col_candidates = ["O*NET Code", "O_NET_Code", "ONET Code", "ONET_Code", "Code"]
code_col = next((c for c in code_col_candidates if c in df_stats.columns), None)
if code_col is None:
  raise KeyError(f"Cannot find O*NET code column in: {df_stats.columns.tolist()}")
title_col = next((c for c in ["Title", "ONET Title", "O*NET Title"] if c in df_stats.columns), None)

df_stats[code_col] = _norm_code_series(df_stats[code_col])

total_rows = len(df_stats)
mapped_mask = df_stats[code_col].notna()
mapped_rows = int(mapped_mask.sum())
overall_rate = (mapped_rows / total_rows * 100) if total_rows else np.nan

print(f"Total clusters (rows): {total_rows}")
print(f"Mapped clusters:       {mapped_rows} ({overall_rate:.2f}%)")

mapped_df = df_stats.loc[mapped_mask].copy()
if title_col:
  mapped_df["Category"] = mapped_df[code_col].astype(str) + " — " + mapped_df[title_col].astype(str)
else:
  mapped_df["Category"] = mapped_df[code_col].astype(str)

counts = mapped_df["Category"].value_counts(dropna=False)
percentages = (counts / mapped_rows * 100).round(2) if mapped_rows else counts * np.nan
category_summary = pd.DataFrame({
  "Category": counts.index,
  "Count": counts.values,
  "Percentage_of_Mapped(%)": percentages.values
})

print("\nCategory distribution (among mapped clusters):")
print(category_summary.to_string(index=False))

if "k" in df_stats.columns:
  by_k = (
    df_stats.assign(mapped=mapped_mask.astype(int))
      .groupby("k")["mapped"]
      .agg(total="count", mapped="sum")
      .reset_index()
)
  by_k["mapping_rate_%"] = (by_k["mapped"] / by_k["total"] * 100).round(2)
  print("\nMapping rate by k:")
  print(by_k.to_string(index=False))
else:
  print("\nNo 'k' column found; skip per-k mapping rate.")

# display top 10 results
TOP_N = 10
print(f"\nTop {min(TOP_N, len(category_summary))} categories by count:")
print(category_summary.head(TOP_N).to_string(index=False))
